# Julia 深度學習：類神經網路模型簡介

## 作業 032：訓練 MLP 學習門牌號碼資料集

訓練一個 MLP 模型來學習門牌號碼資料集。

注意：MLP 模型的能力有限，可能會導致訓練起來效果不佳。

注意：近期 Flux 正在持續更新，請確保您的 Julia 在 v1.3 版以上，以及 Flux 在 v0.10.4 以上或是最新版。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1260


## 讀取資料

呼叫 SVHN2 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。整個資料集的大小約為 1.6 GB，下載時間可能會稍久，請耐心等候。

In [ ]:
train_X, train_y = SVHN2.traindata(Float32)
test_X,  test_y  = SVHN2.testdata(Float32)

This program has requested access to the data dependency SVHN2.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: The Street View House Numbers (SVHN) Dataset
Authors: Yuval Netzer, Tao Wang, Adam Coates, Alessandro Bissacco, Bo Wu, Andrew Y. Ng
Website: http://ufldl.stanford.edu/housenumbers
Format: Cropped Digits (Format 2 on the website)
Note: for non-commercial use only

[Netzer et al., 2011]
    Yuval Netzer, Tao Wang, Adam Coates, Alessandro Bissacco, Bo Wu, Andrew Y. Ng
    "Reading Digits in Natural Images with Unsupervised Feature Learning"
    NIPS Workshop on Deep Learning and Unsupervised Feature Learning 2011

The dataset is split up into three subsets: 73257
digits for training, 26032 digits for testing, and
531131 additional to use as extra training data.

The files are available for download at the official
website linked above. Note that using the data
responsibly and respecting copyright remains you

In [ ]:
train_X = Flux.flatten(train_X)
test_X = Flux.flatten(test_X)
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

In [ ]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

In [ ]:
model = Chain(
  Dense(784, 256, relu),
  Dense(256, 128, relu),
  Dense(128, 10),
  softmax)

In [ ]:
loss(x, y) = logitcrossentropy(model(x), y)

In [ ]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

In [ ]:
evalcb() = @show(test_loss())

In [ ]:
epochs = 20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

In [ ]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

In [ ]:
accuracy(test_X, test_y)